# Fine-tuning Florence2 for VQA (Visual Question Answering) using the Azure ML Python SDK and MLflow

### Overview

Azure ML Workspace is compatible with MLflow and can be used as an MLflow Tracking Server, as described in the following official guide from Microsoft. MLflow provides features such as experiment tracking, model management, and model deployment, allowing you to manage data science and machine learning workflows more efficiently and systematically. Below are the main advantages of using Azure ML and MLflow together.

#### 1. Experiment tracking and management

You can systematically manage the parameters, metrics, and artifacts of all your experiments. Integrating with Azur eML allows you to easily track and manage this information within your Azure ML workspace.

#### 2. Model management

MLflow provides a model registry for model versioning. Integrate with AzureML to systematically manage and deploy all versions of your models. When combined with AzureML's deployment capabilities, models can be easily deployed to a variety of environments (e.g. Azure Kubernetes Service, Azure Container Instances).

#### 3. Reproducibility and collaboration

MLflow records the parameters and environment of every experiment, so you can accurately reproduce the experiment. This is very useful when you need to redo the same experiment across collaborating team members, or when you need to rerun an experiment at a later date.

#### 4. CI/CD integration

MLflow makes it easy to implement continuous integration (CI) and continuous deployment (CD) of machine learning models. Integrate with Azure DevOps or GitHub Actions to automatically run training, validation, and deployment processes as model changes occur.

When training a model with Hugging Face's Trainer API, if you specify `report_to="azure_ml"`, basic indicators will be automatically logged without any additional code. Of course, you can freely log custom indicators using Bring Your Own Script like the conventional method, but Azure ML's basic logging function is also excellent, so try using it as a baseline.

[Note] Please use `Python 3.10 - SDK v2 (azureml_py310_sdkv2)` conda environment.


## Load config file

---


In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
lab_prep_dir = os.getcwd().split("SLMWorkshopCN")[0] + "SLMWorkshopCN/0_lab_preparation"
sys.path.append(os.path.abspath(lab_prep_dir))

from common import check_kernel
check_kernel()

Kernel: python31014jvsc74a57bd01f90a0206bde5cf3732dab79adbbcc7570d5fab64b89fc69d46a8fe33664a709


In [2]:
import os
import yaml
from logger import logger
from datetime import datetime

snapshot_date = datetime.now().strftime("%Y-%m-%d")

with open("config_prd.yml") as f:
    d = yaml.load(f, Loader=yaml.FullLoader)

AZURE_SUBSCRIPTION_ID = d["config"]["AZURE_SUBSCRIPTION_ID"]
AZURE_RESOURCE_GROUP = d["config"]["AZURE_RESOURCE_GROUP"]
AZURE_WORKSPACE = d["config"]["AZURE_WORKSPACE"]
AZURE_DATA_NAME = d["config"]["AZURE_DATA_NAME"]
DATA_DIR = d["config"]["DATA_DIR"]
CLOUD_DIR = d["config"]["CLOUD_DIR"]
HF_MODEL_NAME_OR_PATH = d["config"]["HF_MODEL_NAME_OR_PATH"]
IS_DEBUG = d["config"]["IS_DEBUG"]
USE_LOWPRIORITY_VM = d["config"]["USE_LOWPRIORITY_VM"]

azure_env_name = d["train"]["azure_env_name"]
azure_compute_cluster_name = d["train"]["azure_compute_cluster_name"]
azure_compute_cluster_size = d["train"]["azure_compute_cluster_size"]

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(CLOUD_DIR, exist_ok=True)

logger.info("===== 0. Azure ML Training Info =====")
logger.info(f"AZURE_SUBSCRIPTION_ID={AZURE_SUBSCRIPTION_ID}")
logger.info(f"AZURE_RESOURCE_GROUP={AZURE_RESOURCE_GROUP}")
logger.info(f"AZURE_WORKSPACE={AZURE_WORKSPACE}")
logger.info(f"AZURE_DATA_NAME={AZURE_DATA_NAME}")
logger.info(f"DATA_DIR={DATA_DIR}")
logger.info(f"CLOUD_DIR={CLOUD_DIR}")
logger.info(f"HF_MODEL_NAME_OR_PATH={HF_MODEL_NAME_OR_PATH}")
logger.info(f"IS_DEBUG={IS_DEBUG}")
logger.info(f"USE_LOWPRIORITY_VM={USE_LOWPRIORITY_VM}")

logger.info(f"azure_env_name={azure_env_name}")
logger.info(f"azure_compute_cluster_name={azure_compute_cluster_name}")
logger.info(f"azure_compute_cluster_size={azure_compute_cluster_size}")

2025-02-20 14:34:33,053 - logger - INFO - ===== 0. Azure ML Training Info =====
2025-02-20 14:34:33,054 - logger - INFO - AZURE_SUBSCRIPTION_ID=49aee8bf-3f02-464f-a0ba-e3467e7d85e2
2025-02-20 14:34:33,055 - logger - INFO - AZURE_RESOURCE_GROUP=rg-slmwrkshp_9
2025-02-20 14:34:33,055 - logger - INFO - AZURE_WORKSPACE=mlw-pgwgybluulpec
2025-02-20 14:34:33,056 - logger - INFO - AZURE_DATA_NAME=lgds-sftflorence241201
2025-02-20 14:34:33,057 - logger - INFO - DATA_DIR=./dataset
2025-02-20 14:34:33,057 - logger - INFO - CLOUD_DIR=./cloud
2025-02-20 14:34:33,058 - logger - INFO - HF_MODEL_NAME_OR_PATH=microsoft/Florence-2-base-ft
2025-02-20 14:34:33,059 - logger - INFO - IS_DEBUG=True
2025-02-20 14:34:33,059 - logger - INFO - USE_LOWPRIORITY_VM=False
2025-02-20 14:34:33,060 - logger - INFO - azure_env_name=llm-finetuning-florence-2024-11-05
2025-02-20 14:34:33,060 - logger - INFO - azure_compute_cluster_name=gpu-h100
2025-02-20 14:34:33,061 - logger - INFO - azure_compute_cluster_size=Standard

<br>

## 1. Dataset preparation

---


We store datasets from the HuggingFace hub on shared storage because storing them in the root can run out of space.

已经把数据保持在./dataset中，从github取下来即可，可以不跑这一节。


In [ ]:
import torch
from pathlib import Path
from datasets import load_dataset

curr_dir = Path.cwd()
model_cache_dir = os.path.join(curr_dir.parent.parent, "model")
dataset_cache_dir = os.path.join(curr_dir.parent.parent, "dataset")

dataset = load_dataset("HuggingFaceM4/DocumentVQA", cache_dir=dataset_cache_dir)

In [ ]:
if IS_DEBUG:
    dataset["train"] = dataset["train"].select(range(1000))
    dataset["validation"] = dataset["validation"].select(range(200))
    dataset["test"] = dataset["test"].select(range(200))

In [ ]:
dataset.save_to_disk(DATA_DIR)

In [ ]:
# from local disk
#
# import torch
# from pathlib import Path
# from datasets import load_dataset

# curr_dir = Path.cwd()
# dataset_dir = os.path.join(curr_dir, "dataset_tmp")
# dataset = load_dataset(dataset_dir)

# dataset["train"] = dataset["train"].select(range(16))
# dataset["validation"] = dataset["validation"].select(range(8))
# dataset["test"] = dataset["test"].select(range(8))
# dataset.save_to_disk(DATA_DIR)


/home/luogang/anaconda3/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Saving the dataset (1/1 shards): 100%|██████████| 8/8 [00:00<00:00, 309.37 examples/s]


<br>

## 2. Training preparation

---

### 2.1. Configure workspace details

To connect to a workspace, we need identifying parameters - a subscription, a resource group, and a workspace name. We will use these details in the MLClient from azure.ai.ml to get a handle on the Azure Machine Learning workspace we need. We will use the default Azure authentication for this hands-on.


In [ ]:
# import required libraries
import time
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, Input
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml import command
from azure.ai.ml.entities import Data, Environment, BuildContext
from azure.ai.ml.entities import Model
from azure.ai.ml import Input
from azure.ai.ml import Output
from azure.ai.ml.constants import AssetTypes
from azure.core.exceptions import ResourceNotFoundError, ResourceExistsError

credential = DefaultAzureCredential()
ml_client = MLClient(
    credential, AZURE_SUBSCRIPTION_ID, AZURE_RESOURCE_GROUP, AZURE_WORKSPACE
)

# The code below may conflict with AI Foundry as of February 2025.
# ml_client = None
# try:
#     ml_client = MLClient.from_config(credential)
# except Exception as ex:
#     print(ex)
#     ml_client = MLClient(credential, AZURE_SUBSCRIPTION_ID, AZURE_RESOURCE_GROUP, AZURE_WORKSPACE)
print(ml_client)

### 2.2. Create AzureML environment and data

Azure ML defines containers (called environment asset) in which your code will run. We can use the built-in environment or build a custom environment (Docker container, conda).
This hands-on uses conda yaml.

Training data can be used as a dataset stored in the local development environment, but can also be registered as AzureML data.


In [5]:
from azure.ai.ml.entities import Environment, BuildContext


def get_or_create_environment_asset(
    ml_client, env_name, conda_yml="cloud/conda.yml", update=False
):

    try:
        latest_env_version = max(
            [int(e.version) for e in ml_client.environments.list(name=env_name)]
        )
        if update:
            raise ResourceExistsError(
                "Found Environment asset, but will update the Environment."
            )
        else:
            env_asset = ml_client.environments.get(
                name=env_name, version=latest_env_version
            )
            print(f"Found Environment asset: {env_name}. Will not create again")
    except (ResourceNotFoundError, ResourceExistsError) as e:
        print(f"Exception: {e}")
        env_docker_image = Environment(
            image="mcr.microsoft.com/azureml/curated/acft-hf-nlp-gpu:latest",
            conda_file=conda_yml,
            name=env_name,
            description="Environment created for llm fine-tuning.",
        )
        env_asset = ml_client.environments.create_or_update(env_docker_image)
        print(f"Created Environment asset: {env_name}")

    return env_asset


def get_or_create_docker_environment_asset(
    ml_client, env_name, docker_dir, update=False
):

    try:
        latest_env_version = max(
            [int(e.version) for e in ml_client.environments.list(name=env_name)]
        )
        if update:
            raise ResourceExistsError(
                "Found Environment asset, but will update the Environment."
            )
        else:
            env_asset = ml_client.environments.get(
                name=env_name, version=latest_env_version
            )
            print(f"Found Environment asset: {env_name}. Will not create again")
    except (ResourceNotFoundError, ResourceExistsError) as e:
        print(f"Exception: {e}")
        env_docker_image = Environment(
            build=BuildContext(path=docker_dir),
            name=env_name,
            description="Environment created from a Docker context.",
        )
        env_asset = ml_client.environments.create_or_update(env_docker_image)
        print(f"Created Environment asset: {env_name}")

    return env_asset


def get_or_create_data_asset(ml_client, data_name, data_local_dir, update=False):

    try:
        latest_data_version = max(
            [int(d.version) for d in ml_client.data.list(name=data_name)]
        )
        if update:
            raise ResourceExistsError("Found Data asset, but will update the Data.")
        else:
            data_asset = ml_client.data.get(name=data_name, version=latest_data_version)
            print(f"Found Data asset: {data_name}. Will not create again")
    except (ResourceNotFoundError, ResourceExistsError) as e:
        data = Data(
            path=data_local_dir,
            type=AssetTypes.URI_FOLDER,
            description=f"{data_name} for fine tuning",
            tags={"FineTuningType": "Instruction", "Language": "En"},
            name=data_name,
        )
        data_asset = ml_client.data.create_or_update(data)
        print(f"Created Data asset: {data_name}")

    return data_asset

In [6]:
env = get_or_create_docker_environment_asset(
    ml_client, azure_env_name, docker_dir=f"{CLOUD_DIR}/train", update=False
)
data = get_or_create_data_asset(
    ml_client, AZURE_DATA_NAME, data_local_dir=DATA_DIR, update=False
)

Found Environment asset: llm-finetuning-florence-2024-11-05. Will not create again


Uploading dataset (9.8 MBs): 100%|██████████| 9795704/9795704 [01:55<00:00, 85105.75it/s] 




Created Data asset: lgds-sftflorence241201


### 2.3. Training script


In [ ]:
!pygmentize src_train/train_mlflow.py

<br>

## 3. Training

---

### 3.1. Create the compute cluster


In [7]:
from azure.ai.ml.entities import AmlCompute

### Create the compute cluster
try:
    compute = ml_client.compute.get(azure_compute_cluster_name)
    print("The compute cluster already exists! Reusing it for the current run")
except Exception as ex:
    print(
        f"Looks like the compute cluster doesn't exist. Creating a new one with compute size {azure_compute_cluster_size}!"
    )
    try:
        print("Attempt #1 - Trying to create a dedicated compute")
        tier = "LowPriority" if USE_LOWPRIORITY_VM else "Dedicated"
        compute = AmlCompute(
            name=azure_compute_cluster_name,
            size=azure_compute_cluster_size,
            tier=tier,
            max_instances=1,  # For multi node training set this to an integer value more than 1
        )
        ml_client.compute.begin_create_or_update(compute).wait()
    except Exception as e:
        print("Error")

The compute cluster already exists! Reusing it for the current run


### 3.2. Start training job

The `command` allows user to configure the following key aspects.

-   `inputs` - This is the dictionary of inputs using name value pairs to the command.
    -   `type` - The type of input. This can be a `uri_file` or `uri_folder`. The default is `uri_folder`.
    -   `path` - The path to the file or folder. These can be local or remote files or folders. For remote files - http/https, wasb are supported.
        -   Azure ML `data`/`dataset` or `datastore` are of type `uri_folder`. To use `data`/`dataset` as input, you can use registered dataset in the workspace using the format '<data_name>:<version>'. For e.g Input(type='uri_folder', path='my_dataset:1')
    -   `mode` - Mode of how the data should be delivered to the compute target. Allowed values are `ro_mount`, `rw_mount` and `download`. Default is `ro_mount`
-   `code` - This is the path where the code to run the command is located
-   `compute` - The compute on which the command will run. You can run it on the local machine by using `local` for the compute.
-   `command` - This is the command that needs to be run
    in the `command` using the `${{inputs.<input_name>}}` expression. To use files or folders as inputs, we can use the `Input` class. The `Input` class supports three parameters:
-   `environment` - This is the environment needed for the command to run. Curated (built-in) or custom environments from the workspace can be used.
-   `instance_count` - Number of nodes. Default is 1.
-   `distribution` - Distribution configuration for distributed training scenarios. Azure Machine Learning supports PyTorch, TensorFlow, and MPI-based distributed.


In [ ]:
from azure.ai.ml import command
from azure.ai.ml import Input
from azure.ai.ml.entities import ResourceConfiguration

job = command(
    inputs=dict(
        # train_dir=Input(type="uri_folder", path=DATA_DIR), # Get data from local path
        train_dir=Input(path=f"{AZURE_DATA_NAME}@latest"),  # Get data from Data asset
        epoch=d["train"]["epoch"],
        train_batch_size=d["train"]["train_batch_size"],
        eval_batch_size=d["train"]["eval_batch_size"],
        model_dir=d["train"]["model_dir"],
    ),
    code="./src_train",  # local path where the code is stored
    compute=azure_compute_cluster_name,
    command="python train_mlflow.py --train_dir ${{inputs.train_dir}} --epochs ${{inputs.epoch}} --train_batch_size ${{inputs.train_batch_size}} --eval_batch_size ${{inputs.eval_batch_size}} --model_dir ${{inputs.model_dir}}",
    # environment="azureml://registries/azureml/environments/acft-hf-nlp-gpu/versions/77", # Use built-in Environment asset
    environment=f"{azure_env_name}@latest",
    distribution={
        "type": "PyTorch",
        "process_count_per_instance": 1,  # For multi-gpu training set this to an integer value more than 1
    },
)
returned_job = ml_client.jobs.create_or_update(job)
ml_client.jobs.stream(returned_job.name)

RunId: olive_zoo_nvv1bybcgp
Web View: https://ml.azure.com/runs/olive_zoo_nvv1bybcgp?wsid=/subscriptions/49aee8bf-3f02-464f-a0ba-e3467e7d85e2/resourcegroups/rg-slmwrkshp_9/workspaces/mlw-pgwgybluulpec

Streaming azureml-logs/20_image_build_log.txt

======Starting Image Build on Serverless Compute======
The run ID for the image build on serverless compute is imgbldrun_4f51fff
Additional logs for the run: https://ml.azure.com/experiments/id/prepare_image/runs/imgbldrun_4f51fff?wsid=/subscriptions/49aee8bf-3f02-464f-a0ba-e3467e7d85e2/resourcegroups/rg-slmwrkshp_9/workspaces/mlw-pgwgybluulpec&tid=16b3c013-d300-468d-ac64-7eda0820b6d3
2025-02-20T06:31:28: Logging into Docker registry: f7c27ee9fb96407c9b8fa5c76209316e.azurecr.io
2025-02-20T06:31:28: WARNING! Using --password via the CLI is insecure. Use --password-stdin.
2025-02-20T06:31:29: WARNING! Your password will be stored unencrypted in /root/.docker/config.json.
2025-02-20T06:31:29: Configure a credential helper to remove this warning

In [ ]:
display(returned_job)

In [ ]:
# check if the `trained_model` output is available
job_name = returned_job.name

In [ ]:
%store job_name

<br>

## 4. (Optional) Create model asset and get fine-tuned LLM to local folder

---

### 4.1. Create model asset


In [ ]:
def get_or_create_model_asset(
    ml_client,
    model_name,
    job_name,
    model_dir="outputs",
    model_type="custom_model",
    update=False,
):

    try:
        latest_model_version = max(
            [int(m.version) for m in ml_client.models.list(name=model_name)]
        )
        if update:
            raise ResourceExistsError("Found Model asset, but will update the Model.")
        else:
            model_asset = ml_client.models.get(
                name=model_name, version=latest_model_version
            )
            print(f"Found Model asset: {model_name}. Will not create again")
    except (ResourceNotFoundError, ResourceExistsError) as e:
        print(f"Exception: {e}")
        model_path = f"azureml://jobs/{job_name}/outputs/artifacts/paths/{model_dir}/"
        run_model = Model(
            name=model_name,
            path=model_path,
            description="Model created from run.",
            type=model_type,  # mlflow_model, custom_model, triton_model
        )
        model_asset = ml_client.models.create_or_update(run_model)
        print(f"Created Model asset: {model_name}")

    return model_asset

In [ ]:
azure_model_name = d["serve"]["azure_model_name"]
model_dir = d["train"]["model_dir"]
model = get_or_create_model_asset(
    ml_client,
    azure_model_name,
    job_name,
    model_dir,
    model_type="custom_model",
    update=False,
)

### 4.2. Get fine-tuned LLM to local folder

You can copy it to your local directory to perform inference or serve the model in Azure environment. (e.g., real-time endpoint)


In [ ]:
!rm -rf {local_model_dir}

In [ ]:
# Download the model (this is optional)
local_model_dir = "./artifact_downloads"
os.makedirs(local_model_dir, exist_ok=True)

ml_client.models.download(
    name=azure_model_name, download_path=local_model_dir, version=model.version
)